In [10]:
import sys
sys.path.append("./Preprocessing")
sys.path.append("./Graph")

import pandas as pd
import torch
from feature_engineering import FeatureEngineering
from Resampling import Resampling
from graph_construction import GraphConstruction

In [11]:
dataset = pd.read_csv('C:/Users/ruben/OneDrive/Desktop/Datasets/original_dataset.csv')

# Apply feature engineering on the dataset
fe = FeatureEngineering(dataset)
processed_dataset = fe.apply_feature_engineering()

# Apply resampling on the dataset
resampler = Resampling(processed_dataset, test_size=0.4, random_state=42)
final_dataset = resampler.apply_resampling()

Feature engineering completed.
Fraud rate in training set before resampling: 0.54%
Fraud rate in testing set: 0.62%
Fraud rate in training set after resampling: 50.00%
Fraud rate in testing set after resampling: 0.62%
Length of training set: 953192
Length of testing set: 198889


In [12]:
final_dataset.head()

,index,category,gender,city,state,job,hour,day,month,weekday,age,distance_km,hours_diff_bet_trans,amt_log,is_fraud,cc_user,merchant_num
0,0,0.625724,0,0.548841,0.281046,0.261257,0.956522,0.633333,0.090909,0.500000,0.463415,0.404668,0.00000,0.652271,1,533.0,202.0
1,1,0.551288,1,0.369519,0.880095,0.318870,0.956522,0.133333,0.909091,0.166667,0.426829,0.367390,0.05914,0.056819,0,88.0,125.0
2,2,0.795682,0,0.364171,0.173886,0.581867,0.913043,0.833333,0.454545,0.333333,0.353659,0.436029,0.00000,0.129986,0,424.0,277.0
3,3,0.249912,1,0.181640,0.254662,0.422534,1.000000,0.366667,0.818182,0.833333,0.597561,0.581854,0.00000,0.625768,1,394.0,173.0
4,4,0.912004,1,0.464171,0.172853,0.634261,0.043478,0.200000,0.363636,0.500000,0.414634,0.377778,0.00000,0.526146,1,565.0,215.0


In [13]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [14]:
graph_constructor = GraphConstruction(final_dataset)
data = graph_constructor.apply_graph_construction()

Fraud Percentage in Train Mask: 50.01%
Fraud Percentage in Test Mask: 13.62%
Fraud Percentage in Val Mask: 0.64%


ValueError: 'edge_index' of edge type ('client', 'pays', 'transaction') contains larger source indices than the number of nodes (983) of this node type in 'HeteroData' (found 983)